In [1]:
import matplotlib as mpl
mpl.use('TkAgg')

In [5]:
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button, RadioButtons, TextBox
import tkinter as tk
from tkinter import filedialog
%matplotlib inline
%matplotlib qt
#bug: if I start with inline then change it to qt it's fine...otherwise error if starting qt

In [3]:
def load_process_hic_matrix(infilename):
    x = np.genfromtxt(infilename, delimiter='\t', skip_header=1)
    x = x[:,1:] #take of first row which is generally bin names
    # Next two lines take care of 0's and nan's by setting to dummy value (typically 0.1)
    dummy_val = 0.1
    upper_trim = 99
    x[x == 0] = dummy_val
    x[np.isnan(x)] = dummy_val
    top = np.percentile(x, upper_trim)
    x[x > top] = top
    x = np.log(x)
    x = (x - np.min(x)) / (np.max(x) - np.min(x)) * 1000 # scales to 1-255 betwixt min and max values.
    return(x)

In [55]:
#def viewer(folder):
#for 500 bp: 1000 bins 250,000 bp step

folder = '/Users/MStadler/Bioinformatics/Projects/HiC/data_2018/viewer_data/HiC-53_54_200'
save_folder = '/Users/MStadler/Desktop/'

class Position:
    def __init__(self):
        Position.chr = "2R"
        Position.L = 9000000
        Position.cmap = []
        Position.step_size = 100000
        Position.window_size = 200000

#combine two colormaps
def combine_colormaps(cm1, cm2):
    colors1 = cm1(np.linspace(0., 1, 128))
    colors2 = cm2(np.linspace(0, 1, 128))
    colors = np.vstack((colors1, colors2))
    return(mpl.colors.LinearSegmentedColormap.from_list('my_colormap', colors))

#general function to update genome location
def load_chr_pos():
    posL = pos.L
    posR = posL + pos.window_size
    #print('_'.join([chromosome, L, R]))
    new_file = os.path.join(folder, '_'.join([pos.chr, str(posL), str(posR)]) + '_' + binsize.value_selected + '.txt')
    print(new_file)
    if (os.path.isfile(new_file) or os.path.isfile(new_file + '.gz')): 
        new_img = load_process_hic_matrix(new_file)
        img.set_data(new_img)
        #txt1.set_text(str(posL))
        L_Mb = float(posL) / 1e6
        R_Mb = float(posR) / 1e6
        img.set_extent([L_Mb, R_Mb, R_Mb, L_Mb])
        tloc.set_val(str(L_Mb))
        ax.set_title(pos.chr + ': ' + str(L_Mb) + '-' + str(R_Mb) + ' Mb', fontsize=14)
        img.set_cmap(pos.cmap)
        fig.canvas.draw_idle()
    else:
        ax.set_title('Position is out of bounds!', fontsize=14)


pos = Position()
pos.cmap = combine_colormaps(plt.cm.Blues_r, plt.cm.YlOrBr)



axcolor="white"
fig, ax = plt.subplots()
plt.subplots_adjust(left=0.15, bottom=0.25, right = 0.95, top = 0.95)

img = plt.imshow(np.zeros((100,100)), vmin=0, vmax=1000, cmap=pos.cmap, interpolation="none")
#plt.axis([0, 1, -10, 10])

#######################################################################
# Simple mouse click function to store coordinates
coords = []
def onclick(event):
    if(event.inaxes == ax):
        global coords
        coords.append((pos.chr, event.xdata, event.ydata))
        print((pos.chr, event.xdata, event.ydata))

cid = fig.canvas.mpl_connect('button_press_event', onclick)



#######################################################################

#colormap    
ax_colors = plt.axes([0.025, 0.3, 0.10, 0.15])
colors = RadioButtons(ax_colors, ('BlOr', 'Rd', 'RdGy'), active=0)

def update_colors(selection):
    if (selection == 'BlOr'):
        colormap = combine_colormaps(plt.cm.Blues_r, plt.cm.YlOrBr)
    elif (selection == 'Rd'):
        colormap = plt.cm.Reds
    elif (selection == 'RdGy'):
        colormap = plt.cm.RdGy_r
    pos.cmap = colormap
    img.set_cmap(pos.cmap)
    fig.canvas.draw_idle()   
colors.on_clicked(update_colors)

#######################################################################        

#chromosome radio buttons
ax_chrom = plt.axes([0.025, 0.5, 0.05, 0.15])
chrom = RadioButtons(ax_chrom, ('2L', '2R', '3L', '3R', 'X'), active=0)

def update_chromosome(selection):
    pos.chr = selection
    load_chr_pos()
chrom.on_clicked(update_chromosome)

#######################################################################

#bin size radio buttons
ax_binsize = plt.axes([0.025, 0.8, 0.05, 0.15])
binsize = RadioButtons(ax_binsize, ('500', '1000', '2000', '4000'), active=0)

def update_binsize(selection):
    print(selection)
    pos.step_size = int(selection) * 200
    pos.window_size = int(selection) * 400
    pos.L = int(pos.L / pos.step_size) * pos.step_size #finds nearest position that is a valid start position for this bin size
    load_chr_pos()
binsize.on_clicked(update_binsize)

#######################################################################

#save figure button
ax_savefig = plt.axes([0.025, 0.7, 0.07, 0.05]) #left, bottom, width, height
savefig = Button(ax_savefig, 'Save', color="orange")

def execute_savefig(selection):
    file_path = os.path.join(save_folder, (pos.chr + '_' + str(pos.L) + '.png'))
    mpl.image.imsave(file_path, img.get_array(), vmin=scontrastlower.val, vmax=scontrastupper.val, cmap = pos.cmap)
savefig.on_clicked(execute_savefig)

#######################################################################

#Axis position textbox
ax_position_text = plt.axes([0.55, 0.03, 0.05, 0.05], facecolor=axcolor)
tloc = TextBox(ax_position_text, 'Mb')

def location_text_callback(text):
    pos_entered = int(float(text)* 1000000) #position in megabases
    if (pos_entered % pos.step_size == 0):
        pos.L = pos_entered
        load_chr_pos()
tloc.on_submit(location_text_callback)

#######################################################################

#contrast adjustment sliders   
ax_contrast_lower = plt.axes([0.25, 0.1, 0.65, 0.02], facecolor=axcolor)
ax_contrast_upper = plt.axes([0.25, 0.15, 0.65, 0.02], facecolor=axcolor)
scontrastlower = Slider(ax_contrast_lower, 'Lower', 0, 1000, valinit=0, valstep=1, valfmt="%1.0f", fill=False, edgecolor="blue", linewidth=1)
scontrastupper = Slider(ax_contrast_upper, 'Upper', 0, 1000, valinit=1000, valstep=1, valfmt="%1.0f", fill=False, edgecolor="orange", linewidth=1)

def update_contrast(val):
    lower = int(scontrastlower.val)
    img.set_norm(mpl.colors.Normalize(vmin=scontrastlower.val, vmax=scontrastupper.val))
    fig.canvas.draw_idle()
scontrastlower.on_changed(update_contrast)
scontrastupper.on_changed(update_contrast)

#######################################################################

#left and right scroll buttons
ax_200kb_L = plt.axes([0.45, 0.03, 0.05, 0.05])
ax_200kb_R = plt.axes([0.65, 0.03, 0.05, 0.05])
ax_1Mb_L = plt.axes([0.35, 0.03, 0.05, 0.05])
ax_1Mb_R = plt.axes([0.75, 0.03, 0.05, 0.05])

b200L = Button(ax_200kb_L, '<', color="dodgerblue", hovercolor="darkorange")
b200R = Button(ax_200kb_R, '>', color="dodgerblue", hovercolor="darkorange")
b1MbL = Button(ax_1Mb_L, '<<', color="dodgerblue", hovercolor="darkorange")
b1MbR = Button(ax_1Mb_R, '>>', color="dodgerblue", hovercolor="darkorange")

def update_b200L(click):
    pos.L = int(pos.L - pos.step_size)
    load_chr_pos()
b200L.on_clicked(update_b200L)

def update_b200R(click):
    pos.L = int(pos.L + pos.step_size)
    load_chr_pos()
b200R.on_clicked(update_b200R)

def update_b1MbL(click):
    pos.L = int(pos.L - pos.step_size)
    load_chr_pos()
b1MbL.on_clicked(update_b1MbL)

def update_b1MbR(click):
    pos.L = int(pos.L + pos.step_size)
    load_chr_pos()
b1MbR.on_clicked(update_b1MbR)

load_chr_pos()
#plt.show()

#viewer('/Users/MStadler/Bioinformatics/Projects/HiC/data_2018/viewer_data/HiC_53_54')


/Users/MStadler/Bioinformatics/Projects/HiC/data_2018/viewer_data/HiC-53_54_200/2R_9000000_9200000_500.txt
/Users/MStadler/Bioinformatics/Projects/HiC/data_2018/viewer_data/HiC-53_54_200/2R_9000000_9200000_500.txt
1000
/Users/MStadler/Bioinformatics/Projects/HiC/data_2018/viewer_data/HiC-53_54_200/2R_9000000_9400000_1000.txt
/Users/MStadler/Bioinformatics/Projects/HiC/data_2018/viewer_data/HiC-53_54_200/2R_9200000_9600000_1000.txt
/Users/MStadler/Bioinformatics/Projects/HiC/data_2018/viewer_data/HiC-53_54_200/2R_9200000_9600000_1000.txt
/Users/MStadler/Bioinformatics/Projects/HiC/data_2018/viewer_data/HiC-53_54_200/2R_9400000_9800000_1000.txt
/Users/MStadler/Bioinformatics/Projects/HiC/data_2018/viewer_data/HiC-53_54_200/2R_9400000_9800000_1000.txt
/Users/MStadler/Bioinformatics/Projects/HiC/data_2018/viewer_data/HiC-53_54_200/2R_9600000_10000000_1000.txt
/Users/MStadler/Bioinformatics/Projects/HiC/data_2018/viewer_data/HiC-53_54_200/2R_9600000_10000000_1000.txt
/Users/MStadler/Bioinf

In [51]:
mpl.image.imsave('/Users/MStadler/Desktop/test.png',img.get_array())

In [43]:
binsize